<a href="https://colab.research.google.com/github/juliosdutra/FundamentosComputacionais/blob/main/Utilidades/Serie_CSTRs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Série de 3 CSTRs em Regime Permanente**

Considere um sistema com 3 reatores CSTRs em série, todos operando em regime permanente, com volumes iguais e sob as seguintes condições:

* Reação de 1ª ordem: $𝐴 → 𝐵$, com constante cinética $𝑘 = 0.5\ min^{-1}$
* Vazão de alimentação: $𝐹 = 50\ 𝐿/min⁡$
* Volume de cada reator: $𝑉 = 100\ 𝐿$
* Concentração de entrada de A no primeiro reator: $𝐶_{𝐴0}  = 1.0\  𝑚𝑜𝑙/𝐿$

**Objetivo:**


1.   Calcular a concentração de $A$ na saída de cada reator ($𝐶_{𝐴1}, 𝐶_{𝐴2}, 𝐶_{𝐴3}$)
2.  Determinar a conversão acumulada após cada reator.
3.   Simular os reatores para diferentes valores da constante cinética 𝑘.

----


**Modelagem do sistema físico**

* Reator 1: $F(C_{A0}-C_{A1}) - V_1 k_1 C_{A1} = 0$
* Reator 2: $F(C_{A1}-C_{A2}) - V_2 k_2 C_{A2} = 0$
* Reator 3: $F(C_{A2}-C_{A3}) - V_2 k_2 C_{A3} = 0$

Rearranjando para a concentração de saída de cada reator:
* Reator 1: $C_{A1} = \frac{F}{(F +V_1 k_1)} C_{A0}$
* Reator 2: $C_{A2} = \frac{F}{(F +V_2 k_2)} C_{A1}$
* Reator 3: $C_{A3} = \frac{F}{(F +V_3 k_3)} C_{A2}$


**Modelo genérico**
$$C_{A_i} = \frac{F}{(F +V_i k_i)} C_{A_{i-1}}$$

**Vamos de definir um @dataclass para conter os parâmetros de cada reator**

In [ ]:
from dataclasses import dataclass
from typing import Tuple, List

@dataclass
class CSTR_param:
    F: float = 50.0   # Vazão de alimentação (L/min)
    V: float = 100.0  # Volume do reator (L)
    k: float = 0.5    # Constante cinética de 1ª ordem (1/min)

def modelo_CSTR(param: CSTR_param, CA_entrada: float) -> float:
    """
    Modelo de um reator CSTR: calcula concentração de saída.

    Parâmetros:
        param: objeto com F, V, k do reator
        CA_entrada: concentração na entrada (mol/L)

    Retorna:
        CA_saida: concentração na saída (mol/L)
    """
    return (param.F / (param.F + param.V * param.k)) * CA_entrada

def serie_CSTR(param_list: List[CSTR_param], CA_entrada: float) -> Tuple[List[float], List[float]]:
    """
    Simula uma série de reatores CSTR idênticos em regime permanente.

    Parâmetros:
        param: parâmetros do reator (F, V, k)
        CA_entrada: concentração inicial de A (mol/L)
        nReatores: número de reatores em série

    Retorna:
        Lista de concentrações (entrada e saídas)
        Lista de conversões acumuladas
    """
    Conc = [CA_entrada]
    Conv = []

    for param in param_list:
        CA_saida = modelo_CSTR(param, CA_entrada)
        conversao = 1 - CA_saida / Conc[0]  # conversão em relação a CA0
        Conc.append(CA_saida)
        Conv.append(conversao)
        CA_entrada = CA_saida

    return Conc, Conv

In [ ]:
param = 3*[CSTR_param()]  # usa os valores padrão
CA0 = 1.0
conc, conv = serie_CSTR(param, CA0)

for i in range(1, len(conc)):
    print(f"Reator {i}: C_A = {conc[i]:.4f} mol/L | Conversão acumulada = {conv[i-1]:.2%}")

Reator 1: C_A = 0.5000 mol/L | Conversão acumulada = 50.00%
Reator 2: C_A = 0.2500 mol/L | Conversão acumulada = 75.00%
Reator 3: C_A = 0.1250 mol/L | Conversão acumulada = 87.50%


In [ ]:
params = [
    CSTR_param(F=50, V=100, k=0.5),
    CSTR_param(F=50, V=100, k=0.8),
    CSTR_param(F=50, V=100, k=1.5)
]

CA0 = 1.0
conc, conv = serie_CSTR(params, CA0)

for i in range(1, len(conc)):
    print(f"Reator {i}: C_A = {conc[i]:.4f} mol/L | Conversão acumulada = {conv[i-1]:.2%}")

Reator 1: C_A = 0.5000 mol/L | Conversão acumulada = 50.00%
Reator 2: C_A = 0.1923 mol/L | Conversão acumulada = 80.77%
Reator 3: C_A = 0.0481 mol/L | Conversão acumulada = 95.19%
